# PPS2 New Geometry 2024

### Import Libraries

In [ ]:
import ROOT
import pps_hitmaps
from array import array
import pandas
import numpy
from math import ceil

## Example Sensor Layout

### Original (Square Pads; ETL Design)

In [ ]:
original_sensor = pps_hitmaps.RealisticETLSensor()
original_sensor_ti = pps_hitmaps.RealisticETLSensor(PadSpacing=0.01)
fig = original_sensor.preview()

### Hybrid (Needs Interposer)

In [ ]:
hybrid_sensor = pps_hitmaps.PPSHybrid7Sensor()
hybrid_sensor_ti = pps_hitmaps.PPSHybrid7Sensor(PadSpacing=0.01)
fig = hybrid_sensor.preview()

### New (Rectangular Pads with Gaps)

In [ ]:
new_sensor1 = pps_hitmaps.RectangularPadSensor()
new_sensor1_ti = pps_hitmaps.RectangularPadSensor(PadSpacing=0.01)
fig = new_sensor1.preview()

In [ ]:
new_sensor2 = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16)
new_sensor2_ti = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16, PadSpacing=0.01)
fig = new_sensor2.preview()

### New (Rectangular Pads with Gaps)
Vertical orientation for alignment with ETROC diagrams

In [ ]:
new_sensor3 = pps_hitmaps.RectangularPadSensorVertical(NumSmallerCols = 16)
new_sensor3_ti = pps_hitmaps.RectangularPadSensorVertical(NumSmallerCols = 16, PadSpacing=0.01)
fig = new_sensor3.preview()

## Load the hitmaps and make sure everything is ok

In [ ]:
backgroundFlux = 5*10**12
backgroundTitle = "5E12 p/cm^2"
noBackgroundFlux = 200  # This is needed because when making the sensor los probability maps, we were running into numerican stability issues, giving weird results. Adding this small numbers, makes the behaviour consistent
noBackgroundFlux = 0

hitmaps = {
    "196-physics":            pps_hitmaps.PPSHitmap("MarioHitmaps/map196-physics.out", "196", 10, verbose=True, addBackgroundFlux = noBackgroundFlux),
    "196-physics-background": pps_hitmaps.PPSHitmap("MarioHitmaps/map196-physics.out", "196", 10, verbose=True, addBackgroundFlux = backgroundFlux),
    "220-physics":            pps_hitmaps.PPSHitmap("MarioHitmaps/map220-physics.out", "220", 4.25, verbose=True, addBackgroundFlux = noBackgroundFlux),
    "220-physics-background": pps_hitmaps.PPSHitmap("MarioHitmaps/map220-physics.out", "220", 4.25, verbose=True, addBackgroundFlux = backgroundFlux),
    "234-physics":            pps_hitmaps.PPSHitmap("MarioHitmaps/map234-physics.out", "234", 1.25, yStep=0.000025, xStep=0.000025, verbose=True, addBackgroundFlux = noBackgroundFlux),
    "234-physics-background": pps_hitmaps.PPSHitmap("MarioHitmaps/map234-physics.out", "234", 1.25, yStep=0.000025, xStep=0.000025, verbose=True, addBackgroundFlux = backgroundFlux),
    "420-physics":            pps_hitmaps.PPSHitmap("MarioHitmaps/map420-physics.out", "420", 6.6, yMin=-0.025, yMax=0.025, xMax=0.025, verbose=True, addBackgroundFlux = noBackgroundFlux),
    "420-physics-background": pps_hitmaps.PPSHitmap("MarioHitmaps/map420-physics.out", "420", 6.6, yMin=-0.025, yMax=0.025, xMax=0.025, verbose=True, addBackgroundFlux = backgroundFlux),
}

test_hitmap = hitmaps["196-physics"]

print("Validating the hitmap files:")
for map in hitmaps:
    print("  - Map {}".format(map))
    hitmaps[map].validate()


## Fluxmaps

### Create Histograms

In [ ]:
hitmap_histograms = {}

for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr in hitmaps:
        hitmap_histograms[station] = {
            "physics": hitmaps[hitmapStr].getHisto("station{}-physics".format(station), "Station {}".format(station))
        }

        hitmapStr = "{}-physics-background".format(station)
        if hitmapStr in hitmaps:
            hitmap_histograms[station]["background"] = hitmaps[hitmapStr].getHisto("station{}-physics-background".format(station), f"Station {station} with {backgroundTitle} Background")

### Plot Histograms

In [ ]:
canv = ROOT.TCanvas("physics_detectorEdge", "physics_detectorEdge", 1200, 1200)
canv.Divide(2,2)

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    idx += 1
    pad = canv.cd(idx)

    if station not in hitmap_histograms:
        continue
    if "physics" not in hitmap_histograms[station]:
        continue
    hitmapStr = "{}-physics".format(station)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")

    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)
    line[station].Draw("same")
canv.Draw()

In [ ]:
canv = ROOT.TCanvas("background_effect", "background_effect", 1200, 4*600)
canv.Divide(2,4)

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    idx += 1

    if station not in hitmap_histograms:
        continue
    if "physics" not in hitmap_histograms[station]:
        continue
    if "background" not in hitmap_histograms[station]:
        continue

    hitmapStr = "{}-physics".format(station)
    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)

    pad = canv.cd((idx-1)*2+1)
    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)
    hitmap_histograms[station]["physics"].Draw("colz")
    line[station].Draw("same")

    pad = canv.cd((idx-1)*2+2)
    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)
    hitmap_histograms[station]["background"].SetMinimum(backgroundFlux*0.8)
    hitmap_histograms[station]["background"].Draw("colz")
    line[station].Draw("same")
canv.Draw()

### Overlaid Sensors

#### Sensor Positions

In [ ]:
sensorPositions = {
    "196": [
        [(20.61, -15.05)],
    ],
    "220": [
        [(14.86, -10.2)],
    ],
    "234": [
        [(11.86, -3.5)],
    ],
    "420": [
        [(17.21,  0.25)],
    ],
}

xPos196 = sensorPositions['196'][0][0][0]
yPos196 = sensorPositions['196'][0][0][1]
ySensorSize = original_sensor.maxY - original_sensor.minY
doublePositions = {
    "196": [
        [(xPos196, yPos196 - ySensorSize/2), (xPos196, yPos196 + ySensorSize/2)]
    ]
}

#### Original

In [ ]:
ROOT.gStyle.SetPalette()

canv = ROOT.TCanvas("sensorPositionOriginal", "sensorPositionOriginal", 2600, 2600)
canv.Divide(2,2)

sensorLines = {}

sensor = original_sensor

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr not in hitmaps:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")

    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)
    line[station].Draw("same")

    #offsetX = hitmaps[hitmapStr].detectorEdge * 1000 - sensor.minX
    #for xIdx in hitmaps[hitmapStr].ridge:
    #    if hitmaps[hitmapStr].ridge[xIdx]['x'] >= hitmaps[hitmapStr].detectorEdge:
    #        offsetY = hitmaps[hitmapStr].ridge[xIdx]['y']*1000
    #        break

    offsetX,offsetY = sensorPositions[station][0][0]

    sensorLines[station] = {}
    sensorLines[station]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[station]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[station]:
        sensorLines[station][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[station][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 minX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, minY + offsetY)
        sensorLines[station][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)

        sensorLines[station][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[station]:
        sensorLines[station][key].Draw("same")

canv.Draw()

#### Hybrid

In [ ]:
ROOT.gStyle.SetPalette()

canv = ROOT.TCanvas("sensorPositionHybrid", "sensorPositionHybrid", 2600, 2600)
canv.Divide(2,2)

sensorLines = {}

sensor = hybrid_sensor

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr not in hitmaps:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")

    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)
    line[station].Draw("same")

    #offsetX = hitmaps[hitmapStr].detectorEdge * 1000 - sensor.minX
    #for xIdx in hitmaps[hitmapStr].ridge:
    #    if hitmaps[hitmapStr].ridge[xIdx]['x'] >= hitmaps[hitmapStr].detectorEdge:
    #        offsetY = hitmaps[hitmapStr].ridge[xIdx]['y']*1000
    #        break

    offsetX,offsetY = sensorPositions[station][0][0]

    sensorLines[station] = {}
    sensorLines[station]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[station]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[station]:
        sensorLines[station][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[station][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 minX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, minY + offsetY)
        sensorLines[station][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)

        sensorLines[station][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[station]:
        sensorLines[station][key].Draw("same")

canv.Draw()

#### New

In [ ]:
ROOT.gStyle.SetPalette()

canv = ROOT.TCanvas("sensorPositionNew", "sensorPositionNew", 2600, 2600)
canv.Divide(2,2)

sensorLines = {}

sensor = new_sensor2_ti

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr not in hitmaps:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")

    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)
    line[station].Draw("same")

    #offsetX = hitmaps[hitmapStr].detectorEdge * 1000 - sensor.minX
    #for xIdx in hitmaps[hitmapStr].ridge:
    #    if hitmaps[hitmapStr].ridge[xIdx]['x'] >= hitmaps[hitmapStr].detectorEdge:
    #        offsetY = hitmaps[hitmapStr].ridge[xIdx]['y']*1000
    #        break

    offsetX,offsetY = sensorPositions[station][0][0]

    sensorLines[station] = {}
    sensorLines[station]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[station]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                 sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[station]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                                 sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[station]:
        sensorLines[station][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[station][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 minX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)
        sensorLines[station][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                 maxX + offsetX, minY + offsetY)
        sensorLines[station][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                                 maxX + offsetX, maxY + offsetY)

        sensorLines[station][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[station][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[station]:
        sensorLines[station][key].Draw("same")

canv.Draw()

#### Double Positions

In [ ]:
ROOT.gStyle.SetPalette()

canv = ROOT.TCanvas("doubleSensorPosition", "doubleSensorPosition", 2600, 2600)
canv.Divide(2,2)

sensorLines = {}

sensor = original_sensor

idx = 0
line = {}
for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr not in hitmaps:
        continue

    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    if station not in doublePositions:
        continue

    hitmap_histograms[station]["physics"].Draw("colz")

    edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
    yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
    yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
    line[station] = ROOT.TLine(edge,yMin,edge,yMax)
    line[station].SetLineColor(ROOT.kRed)
    line[station].Draw("same")

    #offsetX = hitmaps[hitmapStr].detectorEdge * 1000 - sensor.minX
    #for xIdx in hitmaps[hitmapStr].ridge:
    #    if hitmaps[hitmapStr].ridge[xIdx]['x'] >= hitmaps[hitmapStr].detectorEdge:
    #        offsetY = hitmaps[hitmapStr].ridge[xIdx]['y']*1000
    #        break

    sensorLines[station] = {}
    pos_idx = -1
    for position in doublePositions[station][0]:
        pos_idx += 1
        offsetX,offsetY = position

        sensorLines[station][f"Left_Pos{pos_idx}"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                                   sensor.minX + offsetX, sensor.maxY + offsetY)
        sensorLines[station][f"Right_Pos{pos_idx}"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                                                   sensor.maxX + offsetX, sensor.maxY + offsetY)
        sensorLines[station][f"Top_Pos{pos_idx}"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                                   sensor.maxX + offsetX, sensor.minY + offsetY)
        sensorLines[station][f"Bottom_Pos{pos_idx}"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                                                   sensor.maxX + offsetX, sensor.maxY + offsetY)

        sensorLines[station][f"Left_Pos{pos_idx}"].SetLineColor(ROOT.kRed)
        sensorLines[station][f"Right_Pos{pos_idx}"].SetLineColor(ROOT.kRed)
        sensorLines[station][f"Top_Pos{pos_idx}"].SetLineColor(ROOT.kRed)
        sensorLines[station][f"Bottom_Pos{pos_idx}"].SetLineColor(ROOT.kRed)

        padID = 0
        for pad in sensor.padVec:
            minX = pad.minX
            maxX = pad.maxX
            minY = pad.minY
            maxY = pad.maxY

            sensorLines[station][f"pad{padID}_Left_Pos{pos_idx}"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                                  minX + offsetX, maxY + offsetY)
            sensorLines[station][f"pad{padID}_Right_Pos{pos_idx}"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                                                  maxX + offsetX, maxY + offsetY)
            sensorLines[station][f"pad{padID}_Top_Pos{pos_idx}"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                                  maxX + offsetX, minY + offsetY)
            sensorLines[station][f"pad{padID}_Bottom_Pos{pos_idx}"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                                                  maxX + offsetX, maxY + offsetY)

            sensorLines[station][f"pad{padID}_Left_Pos{pos_idx}"].SetLineColor(ROOT.kBlue)
            sensorLines[station][f"pad{padID}_Right_Pos{pos_idx}"].SetLineColor(ROOT.kBlue)
            sensorLines[station][f"pad{padID}_Top_Pos{pos_idx}"].SetLineColor(ROOT.kBlue)
            sensorLines[station][f"pad{padID}_Bottom_Pos{pos_idx}"].SetLineColor(ROOT.kBlue)

            padID += 1

    for key in sensorLines[station]:
        sensorLines[station][key].Draw("same")

canv.Draw()

## Occupancy Plots

### Get Square Pad Occupancy and plot it

In [ ]:
square_occupancy_graphs = {}

for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr in hitmaps:
        square_occupancy_graphs[station] = {
            "physics": hitmaps[hitmapStr].squarePadOccupancy()
        }

        hitmapStr = "{}-physics-background".format(station)
        if hitmapStr in hitmaps:
            square_occupancy_graphs[station]["background"] = hitmaps[hitmapStr].squarePadOccupancy()

In [ ]:
for station in ["196", "220", "234", "420"]:
    if station not in square_occupancy_graphs:
        continue
    if 'physics' not in square_occupancy_graphs[station]:
        continue
    if 'background' not in square_occupancy_graphs[station]:
        continue

    if "canv" not in square_occupancy_graphs[station]:
        square_occupancy_graphs[station]["canv"] = ROOT.TCanvas(f"square_occupancy_vs_padSize_{station}", f"Station {station} - Occupancy vs Pad Size", 1200, 600)
        square_occupancy_graphs[station]["canv"].Divide(2,1)

        idx = 0
        for hitmap_type in ['physics', 'background']:
            idx += 1
            pad = square_occupancy_graphs[station]["canv"].cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            title = f"Station {station}"
            if hitmap_type == 'background':
                title = f"Station {station} with {backgroundTitle} Background"

            if "frame" not in square_occupancy_graphs[station][hitmap_type]:
                square_occupancy_graphs[station][hitmap_type]["frame"] = ROOT.TH2D(f"frame_square_{station}_{hitmap_type}", title, 100, 10, 10000, 100, 1.0E-5, 1)
                square_occupancy_graphs[station][hitmap_type]["frame"].SetStats(False)
                square_occupancy_graphs[station][hitmap_type]["frame"].GetXaxis().SetTitle("Pad Size [#mum]")
                square_occupancy_graphs[station][hitmap_type]["frame"].GetYaxis().SetTitle("#mu")

            square_occupancy_graphs[station][hitmap_type]["frame"].Draw()

            square_occupancy_graphs[station][hitmap_type]["uniform"].Draw("l same")
            square_occupancy_graphs[station][hitmap_type]["integrate"].Draw("l same")
            square_occupancy_graphs[station][hitmap_type]["points"].SetMarkerStyle(20)
            square_occupancy_graphs[station][hitmap_type]["points"].Draw("p same")

    square_occupancy_graphs[station]["canv"].Draw()

### Get Rectangular Pad Occupancy and plot it

In [ ]:
rect_occupancy_graphs = {}

for station in ["196", "220", "234", "420"]:
    hitmapStr = "{}-physics".format(station)
    if hitmapStr in hitmaps:
        rect_occupancy_graphs[station] = {
            "physics": hitmaps[hitmapStr].rectangularPadOccupancy(xLen=1.3e-3)
        }

        hitmapStr = "{}-physics-background".format(station)
        if hitmapStr in hitmaps:
            rect_occupancy_graphs[station]["background"] = hitmaps[hitmapStr].rectangularPadOccupancy(xLen=1.3e-3)

In [ ]:
for station in ["196", "220", "234", "420"]:
    if station not in rect_occupancy_graphs:
        continue
    if 'physics' not in rect_occupancy_graphs[station]:
        continue
    if 'background' not in rect_occupancy_graphs[station]:
        continue

    if "canv" not in rect_occupancy_graphs[station]:
        rect_occupancy_graphs[station]["canv"] = ROOT.TCanvas(f"rect_occupancy_vs_padSize_{station}", f"Station {station} - Occupancy vs Pad Size", 1200, 600)
        rect_occupancy_graphs[station]["canv"].Divide(2,1)

        idx = 0
        for hitmap_type in ['physics', 'background']:
            idx += 1
            pad = rect_occupancy_graphs[station]["canv"].cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            title = f"Station {station}"
            if hitmap_type == 'background':
                title = f"Station {station} with {backgroundTitle} Background"

            if "frame" not in rect_occupancy_graphs[station][hitmap_type]:
                rect_occupancy_graphs[station][hitmap_type]["frame"] = ROOT.TH2D(f"frame_rect_{station}_{hitmap_type}", title, 100, 10, 10000, 100, 1.0E-5, 1)
                rect_occupancy_graphs[station][hitmap_type]["frame"].SetStats(False)
                rect_occupancy_graphs[station][hitmap_type]["frame"].GetXaxis().SetTitle("Pad Width [#mum]")
                rect_occupancy_graphs[station][hitmap_type]["frame"].GetYaxis().SetTitle("#mu")

            rect_occupancy_graphs[station][hitmap_type]["frame"].Draw()

            rect_occupancy_graphs[station][hitmap_type]["uniform"].Draw("l same")
            rect_occupancy_graphs[station][hitmap_type]["integrate"].Draw("l same")
            rect_occupancy_graphs[station][hitmap_type]["points"].SetMarkerStyle(20)
            rect_occupancy_graphs[station][hitmap_type]["points"].Draw("p same")

    rect_occupancy_graphs[station]["canv"].Draw()

In [ ]:
occupancy_graphs = {
    'square': square_occupancy_graphs,
    'rectangular': rect_occupancy_graphs,
}

## Event Loss Probabilities

### Convert Occupancy to Event Loss Probabilities

In [ ]:
event_loss_probabilities = {}

for pad_type in occupancy_graphs:
    if pad_type not in event_loss_probabilities:
        event_loss_probabilities[pad_type] = {}

    for station in occupancy_graphs[pad_type]:
        if station not in event_loss_probabilities[pad_type]:
            event_loss_probabilities[pad_type][station] = {}

        for hitmap_type in occupancy_graphs[pad_type][station]:
            if hitmap_type == "canv":
                continue
            event_loss_probabilities[pad_type][station][hitmap_type] = pps_hitmaps.occupancyGraphToEventLossProbability(
                occupancy_graphs[pad_type][station][hitmap_type]['integrate']
            )

### Plot Event Loss Probability vs Pad Dimensions

In [ ]:
prob_vs_dim_canvas = {}
prob_vs_dim_objs = {}

drawTimeSteps = [0, 1, 3, 7, 39, 399]

for station in ["196", "220", "234", "420"]:
    do_continue = False
    for pad_type in occupancy_graphs:
        if station not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    if station not in prob_vs_dim_objs:
        prob_vs_dim_objs[station] = {}

    canv = ROOT.TCanvas(f"probability_vs_dimension_{station}", f"Station {station} - Event Loss Probability vs Pad Dimensions", 1200, 1200)
    canv.Divide(2,2)

    idx = 0
    for pad_type in event_loss_probabilities:
        pad_title = "Square Pad"
        x_title = "Pad Size [#mum]"
        if pad_type == "rectangular":
            pad_title = "Rectangular Pad"
            x_title = "Pad Width [#mum]"

        if pad_type not in prob_vs_dim_objs[station]:
            prob_vs_dim_objs[station][pad_type] = {}

        for hitmap_type in event_loss_probabilities[pad_type][station]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" with {backgroundTitle} Background"

            if hitmap_type not in prob_vs_dim_objs[station][pad_type]:
                prob_vs_dim_objs[station][pad_type][hitmap_type] = {}

            frame = ROOT.TH2D(f"frame_{station}_{pad_type}_{hitmap_type}", f"Station {station} {pad_title}{hitmap_title}", 100, 10, 10000, 100, 0.001, 1.1)
            frame.SetStats(False)
            frame.GetXaxis().SetTitle(x_title)
            frame.GetYaxis().SetTitle("Event Loss Probability")

            legend = ROOT.TLegend(0.65,0.1,0.9,0.45)

            prob_vs_dim_objs[station][pad_type][hitmap_type]['frame'] = frame
            prob_vs_dim_objs[station][pad_type][hitmap_type]['legend'] = legend

            idx += 1
            pad = canv.cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            frame.Draw()

            colorOffset = 0
            for timeStep in drawTimeSteps:
                probability = event_loss_probabilities[pad_type][station][hitmap_type][f'prob_timeStep{timeStep}']
                length      = event_loss_probabilities[pad_type][station][hitmap_type]['length']

                x, y = array( 'd' ), array( 'd' )
                for ibin in range(len(length)):
                    x.append(length[ibin])
                    y.append(probability[ibin])
                graph = ROOT.TGraph(len(length), x, y)

                graph.SetLineColor(ROOT.kAzure + colorOffset)
                graph.Draw("l same")
                legend.AddEntry(graph, "#tau #leq {} ns".format((timeStep+1)*25), "l")
                colorOffset += 3

                prob_vs_dim_objs[station][pad_type][hitmap_type][f'graph_timeStep{timeStep}'] = graph

            legend.Draw()

    canv.Draw()
    prob_vs_dim_canvas[station] = canv

### Reference Points for Pad Sizing

#### Pad Size Needed for 10% efficiency with 25 ns deadtime

In [ ]:
table = []
rows = []
columns = []
for station in ["196", "220", "234", "420"]:
    do_continue = False
    for pad_type in occupancy_graphs:
        if station not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    these_columns = []
    this_row = []
    for pad_type in event_loss_probabilities:
        pad_title = "Square"
        if pad_type == "rectangular":
            pad_title = "Rect"

        for hitmap_type in event_loss_probabilities[pad_type][station]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" w/ Bkg"

            these_columns += [f"{pad_title}{hitmap_title}"]

            probability = event_loss_probabilities[pad_type][station][hitmap_type][f'prob_timeStep0']
            length      = event_loss_probabilities[pad_type][station][hitmap_type]['length']

            this_row += [numpy.interp(0.1, probability, length)]

    if columns == []:
        columns = these_columns
    if these_columns != columns:
        print("There is a problem with the ordering")

    table += [this_row]
    rows += [station]

df = pandas.DataFrame(table, columns = columns, index=rows)
print(df)

Units above in $\mu m$

#### Loss Probability for different pad sizes with 25 ns deadtime

In [ ]:
table2 = []
table3 = []
table4 = []
rows = []
columns = []
for station in ["196", "220", "234", "420"]:
    do_continue = False
    for pad_type in occupancy_graphs:
        if station not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    these_columns = []
    this_row2 = []
    this_row3 = []
    this_row4 = []
    for pad_type in event_loss_probabilities:
        pad_title = "Square"
        if pad_type == "rectangular":
            pad_title = "Rect"

        for hitmap_type in event_loss_probabilities[pad_type][station]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" w/ Bkg"

            these_columns += [f"{pad_title}{hitmap_title}"]

            probability = event_loss_probabilities[pad_type][station][hitmap_type][f'prob_timeStep0']
            length      = event_loss_probabilities[pad_type][station][hitmap_type]['length']

            this_row2 += [numpy.interp(1300/2, length, probability)]
            this_row3 += [numpy.interp(1300/3, length, probability)]
            this_row4 += [numpy.interp(1300/4, length, probability)]

    if columns == []:
        columns = these_columns
    if these_columns != columns:
        print("There is a problem with the ordering")

    table2 += [this_row2]
    table3 += [this_row3]
    table4 += [this_row4]
    rows += [station]

df2 = pandas.DataFrame(table2, columns = columns, index=rows)
df3 = pandas.DataFrame(table3, columns = columns, index=rows)
df4 = pandas.DataFrame(table4, columns = columns, index=rows)

print(f"Pad dimension 1/2 of nominal ({1.3/2})")
print(df2)
print()

print(f"Pad dimension 1/3 of nominal ({1.3/3})")
print(df3)
print()

print(f"Pad dimension 1/4 of nominal ({1.3/4})")
print(df4)
print()

## Vertical Shifts needed over time

Assuming 1 year of HL-LHC is $300 fb^{-1}$ and LGAD limit of 2e15 neq/cm^2

In [ ]:
lgad_limit = 2e15
luminosity = 300

### Station 196, no Background

In [ ]:
hitmap_id = "196-physics"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=4,
    )
    canv.Draw()

### Station 196 with Background

In [ ]:
hitmap_id = "196-physics-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=4,
    )
    canv.Draw()

### Station 220, no Background

In [ ]:
hitmap_id = "220-physics"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=4,
    )
    canv.Draw()

### Station 220 with Background

In [ ]:
hitmap_id = "220-physics-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=4,
    )
    canv.Draw()

### Station 234, no Background

In [ ]:
hitmap_id = "234-physics"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=12,
    )
    canv.Draw()

### Station 234 with Background

In [ ]:
hitmap_id = "234-physics-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=12,
    )
    canv.Draw()

### Station 420, no Background

In [ ]:
hitmap_id = "420-physics"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=4,
    )
    canv.Draw()

### Station 420 with Background

In [ ]:
hitmap_id = "420-physics-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        #plotPadCols=3,
        maxNumShifts=5,
    )
    canv.Draw()

### Needed Shifts

In [ ]:
needed_shifts = {
    '196': (0, None),   # No shift needed
    '220': (2, 1.3*2),  # Each shift by 1 standard pad, i.e. 1.3 mm
    '234': (10, 1.3*5), # Each shift by 1/2 standard pad, i.e. 0.625 mm
    '420': (4, 1.3*4),  # Each shift by 1 standard pad, i.e. 1.3 mm
}

### Compute positions with shifts

In [ ]:
for station in needed_shifts:
    if station not in sensorPositions:
        continue
    num_shifts, total_shift = needed_shifts[station]
    if num_shifts == 0 or total_shift is None:
        continue

    center_position_x, center_position_y = sensorPositions[station][0][0]

    positions = []

    start_position_y = center_position_y - total_shift/2
    step_y = total_shift/num_shifts
    for idx in range(num_shifts + 1):
        positions += [(center_position_x, start_position_y + step_y*idx)]

    sensorPositions[station] += [positions]

### Sensor Positions on Hitmap with shifts

#### Station 196

In [ ]:
station = '196'
sensor = original_sensor

ROOT.gStyle.SetPalette()

positions = sensorPositions[station][-1]
num_pos = len(positions)

if num_pos <= 3:
    xPads = num_pos
    yPads = 1
elif num_pos <= 4:
    xPads = 2
    yPads = 2
elif num_pos <= 9:
    xPads = 3
    yPads = ceil(num_pos/3)
else:
    xPads = 4
    yPads = ceil(num_pos/4)

canv = ROOT.TCanvas(f"sensorPositionShifts{station}", f"sensorPositionShifts{station}", xPads*1300, yPads*1300)
canv.Divide(xPads,yPads)

hitmapStr = "{}-physics".format(station)
edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
edgeLine = ROOT.TLine(edge,yMin,edge,yMax)
edgeLine.SetLineColor(ROOT.kRed)

idx = 0
sensorLines = {}
for offsetX, offsetY in positions:
    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")
    edgeLine.Draw("same")

    sensorLines[idx] = {}
    sensorLines[idx]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[idx]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[idx]:
        sensorLines[idx][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[idx][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             minX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             maxX + offsetX, minY + offsetY)
        sensorLines[idx][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)

        sensorLines[idx][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[idx]:
        sensorLines[idx][key].Draw("same")

canv.Draw()


#### Station 220

In [ ]:
station = '220'
sensor = original_sensor

ROOT.gStyle.SetPalette()

positions = sensorPositions[station][-1]
num_pos = len(positions)

if num_pos <= 3:
    xPads = num_pos
    yPads = 1
elif num_pos <= 4:
    xPads = 2
    yPads = 2
elif num_pos <= 9:
    xPads = 3
    yPads = ceil(num_pos/3)
else:
    xPads = 4
    yPads = ceil(num_pos/4)

canv = ROOT.TCanvas(f"sensorPositionShifts{station}", f"sensorPositionShifts{station}", xPads*1300, yPads*1300)
canv.Divide(xPads,yPads)

hitmapStr = "{}-physics".format(station)
edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
edgeLine = ROOT.TLine(edge,yMin,edge,yMax)
edgeLine.SetLineColor(ROOT.kRed)

idx = 0
sensorLines = {}
for offsetX, offsetY in positions:
    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")
    edgeLine.Draw("same")

    sensorLines[idx] = {}
    sensorLines[idx]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[idx]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[idx]:
        sensorLines[idx][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[idx][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             minX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             maxX + offsetX, minY + offsetY)
        sensorLines[idx][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)

        sensorLines[idx][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[idx]:
        sensorLines[idx][key].Draw("same")

canv.Draw()


#### Station 234

In [ ]:
station = '234'
sensor = original_sensor

ROOT.gStyle.SetPalette()

positions = sensorPositions[station][-1]
num_pos = len(positions)

if num_pos <= 3:
    xPads = num_pos
    yPads = 1
elif num_pos <= 4:
    xPads = 2
    yPads = 2
elif num_pos <= 9:
    xPads = 3
    yPads = ceil(num_pos/3)
else:
    xPads = 4
    yPads = ceil(num_pos/4)

canv = ROOT.TCanvas(f"sensorPositionShifts{station}", f"sensorPositionShifts{station}", xPads*1300, yPads*1300)
canv.Divide(xPads,yPads)

hitmapStr = "{}-physics".format(station)
edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
edgeLine = ROOT.TLine(edge,yMin,edge,yMax)
edgeLine.SetLineColor(ROOT.kRed)

idx = 0
sensorLines = {}
for offsetX, offsetY in positions:
    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")
    edgeLine.Draw("same")

    sensorLines[idx] = {}
    sensorLines[idx]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[idx]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[idx]:
        sensorLines[idx][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[idx][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             minX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             maxX + offsetX, minY + offsetY)
        sensorLines[idx][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)

        sensorLines[idx][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[idx]:
        sensorLines[idx][key].Draw("same")

canv.Draw()


#### Station 420

In [ ]:
station = '420'
sensor = original_sensor

ROOT.gStyle.SetPalette()

positions = sensorPositions[station][-1]
num_pos = len(positions)

if num_pos <= 3:
    xPads = num_pos
    yPads = 1
elif num_pos <= 4:
    xPads = 2
    yPads = 2
elif num_pos <= 9:
    xPads = 3
    yPads = ceil(num_pos/3)
else:
    xPads = 4
    yPads = ceil(num_pos/4)

canv = ROOT.TCanvas(f"sensorPositionShifts{station}", f"sensorPositionShifts{station}", xPads*1300, yPads*1300)
canv.Divide(xPads,yPads)

hitmapStr = "{}-physics".format(station)
edge = hitmaps[hitmapStr].detectorEdge * 1000 # Convert to mm for drawing
yMin = hitmaps[hitmapStr].yMin * 1000         # Convert to mm for drawing
yMax = hitmaps[hitmapStr].yMax * 1000         # Convert to mm for drawing
edgeLine = ROOT.TLine(edge,yMin,edge,yMax)
edgeLine.SetLineColor(ROOT.kRed)

idx = 0
sensorLines = {}
for offsetX, offsetY in positions:
    idx += 1
    pad = canv.cd(idx)

    pad.SetTicks()
    pad.SetLogz()
    pad.SetLeftMargin(0.11)
    pad.SetRightMargin(0.16)
    pad.SetTopMargin(0.07)
    pad.SetBottomMargin(0.14)

    hitmap_histograms[station]["physics"].Draw("colz")
    edgeLine.Draw("same")

    sensorLines[idx] = {}
    sensorLines[idx]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.minX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    sensorLines[idx]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                             sensor.maxX + offsetX, sensor.minY + offsetY)
    sensorLines[idx]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                             sensor.maxX + offsetX, sensor.maxY + offsetY)
    for key in sensorLines[idx]:
        sensorLines[idx][key].SetLineColor(ROOT.kRed)

    padID = 0
    for pad in sensor.padVec:
        minX = pad.minX
        maxX = pad.maxX
        minY = pad.minY
        maxY = pad.maxY

        sensorLines[idx][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             minX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)
        sensorLines[idx][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                             maxX + offsetX, minY + offsetY)
        sensorLines[idx][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                             maxX + offsetX, maxY + offsetY)

        sensorLines[idx][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
        sensorLines[idx][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

        padID += 1

    for key in sensorLines[idx]:
        sensorLines[idx][key].Draw("same")

canv.Draw()


## Sensor Maximum Occupancy and Loss Probability

In [ ]:
sensors = {
    "ETL": original_sensor,
    "ETL_TI": original_sensor_ti,
    "Hybrid": hybrid_sensor_ti,
    "Rect": new_sensor1_ti,
    "RectFull": new_sensor2_ti,
}

### Station 196

In [ ]:
station = '196'

center_position_x, center_position_y = sensorPositions[station][0][0]

coverage_shifts = 4
coverage_shift_step = 1.3/4
coverage_positions = []
for step in range(coverage_shifts):
    coverage_positions += [(center_position_x, center_position_y + step*coverage_shift_step)]

hitmap      = hitmaps["{}-physics".format(station)]
hitmap_bckg = hitmaps["{}-physics-background".format(station)]

for sensor_name in sensors:
    sensor = sensors[sensor_name]
    table_data = []
    rows = []
    cols = ["Dose", "Occ", "Loss p", "Pad", "Dose w/ Bckg", "Occ w/ Bckg", "Loss p w/ Bckg", "Pad w/ Bckg"]
    for step in range(coverage_shifts):
        current_position = [(center_position_x, center_position_y + step*coverage_shift_step)]
        sensor.setShifts(current_position)

        # Calculations without Background
        sensor.calculateFlux(hitmap)
        dose      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies, pads = sensor.findMaxOccupancy()
        occupancy = occupancies[0]
        pad = pads[0]
        loss_prob = pps_hitmaps.calcEventLossProb(0, occupancy)

        # Calculations with Background
        sensor.calculateFlux(hitmap_bckg)
        dose_bckg      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies_bckg, pads_bckg = sensor.findMaxOccupancy()
        occupancy_bckg = occupancies_bckg[0]
        pad_bckg = pads_bckg[0]
        loss_prob_bckg = pps_hitmaps.calcEventLossProb(0, occupancy_bckg)

        rows += [f"Offset {step}"]
        table_data += [[dose, occupancy, loss_prob, pad, dose_bckg, occupancy_bckg, loss_prob_bckg, pad_bckg]]

    df = pandas.DataFrame(table_data, columns = cols, index=rows)
    print(f"Sensor {sensor_name}:")
    print(df)
    print()

#### ETL Sensor

In [ ]:
sensor = sensors["ETL"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Trench Isolated ETL Sensor

In [ ]:
sensor = sensors["ETL_TI"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Hybrid Sensor

In [ ]:
sensor = sensors["Hybrid"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Rectangular Sensor

In [ ]:
sensor = sensors["Rect"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Full Rectangular Sensor

In [ ]:
sensor = sensors["RectFull"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

### Station 220

In [ ]:
station = '220'

center_position_x, center_position_y = sensorPositions[station][0][0]

coverage_shifts = 4
coverage_shift_step = 1.3/4
coverage_positions = []
for step in range(coverage_shifts):
    coverage_positions += [(center_position_x, center_position_y + step*coverage_shift_step)]

hitmap      = hitmaps["{}-physics".format(station)]
hitmap_bckg = hitmaps["{}-physics-background".format(station)]

for sensor_name in sensors:
    sensor = sensors[sensor_name]
    table_data = []
    rows = []
    cols = ["Dose", "Occ", "Loss p", "Pad", "Dose w/ Bckg", "Occ w/ Bckg", "Loss p w/ Bckg", "Pad w/ Bckg"]
    for step in range(coverage_shifts):
        current_position = [(center_position_x, center_position_y + step*coverage_shift_step)]
        sensor.setShifts(current_position)

        # Calculations without Background
        sensor.calculateFlux(hitmap)
        dose      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies, pads = sensor.findMaxOccupancy()
        occupancy = occupancies[0]
        pad = pads[0]
        loss_prob = pps_hitmaps.calcEventLossProb(0, occupancy)

        # Calculations with Background
        sensor.calculateFlux(hitmap_bckg)
        dose_bckg      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies_bckg, pads_bckg = sensor.findMaxOccupancy()
        occupancy_bckg = occupancies_bckg[0]
        pad_bckg = pads_bckg[0]
        loss_prob_bckg = pps_hitmaps.calcEventLossProb(0, occupancy_bckg)

        rows += [f"Offset {step}"]
        table_data += [[dose, occupancy, loss_prob, pad, dose_bckg, occupancy_bckg, loss_prob_bckg, pad_bckg]]

    df = pandas.DataFrame(table_data, columns = cols, index=rows)
    print(f"Sensor {sensor_name}:")
    print(df)
    print()

#### ETL Sensor

In [ ]:
sensor = sensors["ETL"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Trench Isolated ETL Sensor

In [ ]:
sensor = sensors["ETL_TI"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Hybrid Sensor

In [ ]:
sensor = sensors["Hybrid"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Rectangular Sensor

In [ ]:
sensor = sensors["Rect"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Full Rectangular Sensor

In [ ]:
sensor = sensors["RectFull"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

### Station 234

In [ ]:
station = '234'

center_position_x, center_position_y = sensorPositions[station][0][0]

coverage_shifts = 4
coverage_shift_step = 1.3/4
coverage_positions = []
for step in range(coverage_shifts):
    coverage_positions += [(center_position_x, center_position_y + step*coverage_shift_step)]

hitmap      = hitmaps["{}-physics".format(station)]
hitmap_bckg = hitmaps["{}-physics-background".format(station)]

for sensor_name in sensors:
    sensor = sensors[sensor_name]
    table_data = []
    rows = []
    cols = ["Dose", "Occ", "Loss p", "Pad", "Dose w/ Bckg", "Occ w/ Bckg", "Loss p w/ Bckg", "Pad w/ Bckg"]
    for step in range(coverage_shifts):
        current_position = [(center_position_x, center_position_y + step*coverage_shift_step)]
        sensor.setShifts(current_position)

        # Calculations without Background
        sensor.calculateFlux(hitmap)
        dose      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies, pads = sensor.findMaxOccupancy()
        occupancy = occupancies[0]
        pad = pads[0]
        loss_prob = pps_hitmaps.calcEventLossProb(0, occupancy)

        # Calculations with Background
        sensor.calculateFlux(hitmap_bckg)
        dose_bckg      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies_bckg, pads_bckg = sensor.findMaxOccupancy()
        occupancy_bckg = occupancies_bckg[0]
        pad_bckg = pads_bckg[0]
        loss_prob_bckg = pps_hitmaps.calcEventLossProb(0, occupancy_bckg)

        rows += [f"Offset {step}"]
        table_data += [[dose, occupancy, loss_prob, pad, dose_bckg, occupancy_bckg, loss_prob_bckg, pad_bckg]]

    df = pandas.DataFrame(table_data, columns = cols, index=rows)
    print(f"Sensor {sensor_name}:")
    print(df)
    print()

#### ETL Sensor

In [ ]:
sensor = sensors["ETL"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Trench Isolated ETL Sensor

In [ ]:
sensor = sensors["ETL_TI"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Hybrid Sensor

In [ ]:
sensor = sensors["Hybrid"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Rectangular Sensor

In [ ]:
sensor = sensors["Rect"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Full Rectangular Sensor

In [ ]:
sensor = sensors["RectFull"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

### Station 420

In [ ]:
station = '420'

center_position_x, center_position_y = sensorPositions[station][0][0]

coverage_shifts = 4
coverage_shift_step = 1.3/4
coverage_positions = []
for step in range(coverage_shifts):
    coverage_positions += [(center_position_x, center_position_y + step*coverage_shift_step)]

hitmap      = hitmaps["{}-physics".format(station)]
hitmap_bckg = hitmaps["{}-physics-background".format(station)]

for sensor_name in sensors:
    sensor = sensors[sensor_name]
    table_data = []
    rows = []
    cols = ["Dose", "Occ", "Loss p", "Pad", "Dose w/ Bckg", "Occ w/ Bckg", "Loss p w/ Bckg", "Pad w/ Bckg"]
    for step in range(coverage_shifts):
        current_position = [(center_position_x, center_position_y + step*coverage_shift_step)]
        sensor.setShifts(current_position)

        # Calculations without Background
        sensor.calculateFlux(hitmap)
        dose      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies, pads = sensor.findMaxOccupancy()
        occupancy = occupancies[0]
        pad = pads[0]
        loss_prob = pps_hitmaps.calcEventLossProb(0, occupancy)

        # Calculations with Background
        sensor.calculateFlux(hitmap_bckg)
        dose_bckg      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies_bckg, pads_bckg = sensor.findMaxOccupancy()
        occupancy_bckg = occupancies_bckg[0]
        pad_bckg = pads_bckg[0]
        loss_prob_bckg = pps_hitmaps.calcEventLossProb(0, occupancy_bckg)

        rows += [f"Offset {step}"]
        table_data += [[dose, occupancy, loss_prob, pad, dose_bckg, occupancy_bckg, loss_prob_bckg, pad_bckg]]

    df = pandas.DataFrame(table_data, columns = cols, index=rows)
    print(f"Sensor {sensor_name}:")
    print(df)
    print()

#### ETL Sensor

In [ ]:
sensor = sensors["ETL"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Trench Isolated ETL Sensor

In [ ]:
sensor = sensors["ETL_TI"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Hybrid Sensor

In [ ]:
sensor = sensors["Hybrid"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Rectangular Sensor

In [ ]:
sensor = sensors["Rect"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

#### Full Rectangular Sensor

In [ ]:
sensor = sensors["RectFull"]

sensor.setShifts(coverage_positions)
sensor.calculateFlux(hitmap)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
sensor.calculateFlux(hitmap_bckg)

In [ ]:
canv, persistance = sensor.plotSensorQuantity("flux")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("occupancy")
canv.Draw()

In [ ]:
canv, persistance = sensor.plotSensorQuantity("loss_probability")
canv.Draw()

## Placeholder

# Testing

In [ ]:
print(test_hitmap.peakUniformPadOccupancy(1.3*10**-3, 1.3*10**-3))

print(test_hitmap.rectangularPadIntegrateScan(2,1e-6,1.3e-3,xLen=1.3e-3))
print(test_hitmap.squarePadIntegrateScan(2,1e-6,1.3e-3))

print(test_hitmap.rectangularPadPeakUniformScan(2,1e-6,1.3e-3,xLen=1.3e-3))
print(test_hitmap.squarePadPeakUniformScan(2,1e-6,1.3e-3))